# Resumen y Comparación de Modelos SEM

Este notebook **consolida y compara** todos los resultados de los modelos de ecuaciones estructurales ejecutados en los notebooks 55-58.

## Modelos Analizados:

**Notebook 55 - Variables Sumadas (8 modelos)**
- Cambio_Op_Sum (4 variables)
- Cambio_Tiempo_Sum (4 variables)

**Notebook 56 - Variables Filtradas (8 modelos)**
- Cambio_Op_Filt (4 variables)
- Cambio_Tiempo_Filt (4 variables)

**Notebook 57 - Variables de Congruencia (4 modelos)**
- CO_Congruente, CO_Incongruente
- CT_Congruente, CT_Incongruente

**Notebook 58 - Por Tipo de Ítem (4 modelos)**
- CO_Total_Progresistas, CO_Total_Conservadores
- CT_Total_Progresistas, CT_Total_Conservadores

**Total: 24 modelos × 2 elecciones = 48 modelos**

## Objetivos:

1. ✅ Consolidar todos los resultados en una tabla maestra
2. ✅ Identificar modelos con mejor ajuste (R² más alto)
3. ✅ Determinar qué índice ideológico "pesa más"
4. ✅ Comparar poder predictivo entre grupos de variables
5. ✅ Generar conclusiones generales del análisis SEM

In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')

from openpyxl import Workbook, load_workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter

print("✓ Librerías cargadas exitosamente")

✓ Librerías cargadas exitosamente


## 1. Cargar Todos los Resultados SEM

In [2]:
# Carpeta de resultados
Carpeta_Resultados = os.path.join(os.getcwd(), '..', 'Data', 'Resultados_SEM')

# Archivos de resultados
archivos_sem = [
    ('Sumadas', 'SEM_Variables_Sumadas'),
    ('Filtradas', 'SEM_Variables_Filtradas'),
    ('Congruencia', 'SEM_Variables_Congruencia'),
    ('Tipo_Item', 'SEM_Por_Tipo_Item')
]

# Diccionario para almacenar resultados
resultados = {
    'Generales': {},
    'Ballotage': {}
}

print("="*70)
print("CARGANDO RESULTADOS DE MODELOS SEM")
print("="*70)

for tipo, nombre_base in archivos_sem:
    print(f"\n📁 {tipo}:")
    
    for eleccion in ['Generales', 'Ballotage']:
        archivo = os.path.join(Carpeta_Resultados, f'{nombre_base}_{eleccion}.xlsx')
        
        if os.path.exists(archivo):
            # Cargar métricas
            df_metricas = pd.read_excel(archivo, sheet_name='Métricas de Ajuste')
            df_coef = pd.read_excel(archivo, sheet_name='Coeficientes')
            
            resultados[eleccion][tipo] = {
                'metricas': df_metricas,
                'coeficientes': df_coef
            }
            
            print(f"  ✓ {eleccion}: {len(df_metricas)} modelos cargados")
        else:
            print(f"  ⚠️  {eleccion}: Archivo no encontrado")

print("\n" + "="*70)
print("✅ RESULTADOS CARGADOS")
print("="*70)

CARGANDO RESULTADOS DE MODELOS SEM

📁 Sumadas:
  ✓ Generales: 8 modelos cargados
  ✓ Ballotage: 8 modelos cargados

📁 Filtradas:
  ⚠️  Generales: Archivo no encontrado
  ⚠️  Ballotage: Archivo no encontrado

📁 Congruencia:
  ⚠️  Generales: Archivo no encontrado
  ⚠️  Ballotage: Archivo no encontrado

📁 Tipo_Item:
  ⚠️  Generales: Archivo no encontrado
  ⚠️  Ballotage: Archivo no encontrado

✅ RESULTADOS CARGADOS


## 2. Consolidar Todas las Métricas en Tabla Maestra

In [3]:
print("\n📊 Consolidando métricas de todos los modelos...")

# Listas para consolidar
metricas_consolidadas_gen = []
metricas_consolidadas_bal = []

for tipo, _ in archivos_sem:
    # Generales
    if tipo in resultados['Generales']:
        df_temp = resultados['Generales'][tipo]['metricas'].copy()
        df_temp['Tipo'] = tipo
        df_temp['Eleccion'] = 'Generales'
        metricas_consolidadas_gen.append(df_temp)
    
    # Ballotage
    if tipo in resultados['Ballotage']:
        df_temp = resultados['Ballotage'][tipo]['metricas'].copy()
        df_temp['Tipo'] = tipo
        df_temp['Eleccion'] = 'Ballotage'
        metricas_consolidadas_bal.append(df_temp)

# Concatenar todo
df_metricas_todas_gen = pd.concat(metricas_consolidadas_gen, ignore_index=True)
df_metricas_todas_bal = pd.concat(metricas_consolidadas_bal, ignore_index=True)
df_metricas_todas = pd.concat([df_metricas_todas_gen, df_metricas_todas_bal], ignore_index=True)

print(f"\n✓ Total de modelos consolidados: {len(df_metricas_todas)}")
print(f"  - Generales: {len(df_metricas_todas_gen)}")
print(f"  - Ballotage: {len(df_metricas_todas_bal)}")


📊 Consolidando métricas de todos los modelos...

✓ Total de modelos consolidados: 16
  - Generales: 8
  - Ballotage: 8


## 3. Top 10 Modelos con Mejor Ajuste (Mayor R²)

In [4]:
print("="*70)
print("TOP 10 MODELOS CON MEJOR AJUSTE (Mayor R²)")
print("="*70)

# Ordenar por R²
top10_general = df_metricas_todas.sort_values('R²', ascending=False).head(10)

print("\n🏆 Top 10 Global (Todas las elecciones):")
print("-"*70)
print(top10_general[['Eleccion', 'Tipo', 'Outcome', 'R²', 'AIC', 'BIC']].to_string(index=False))

# Top 5 por elección
print("\n🏆 Top 5 - GENERALES:")
print("-"*70)
top5_gen = df_metricas_todas_gen.sort_values('R²', ascending=False).head(5)
print(top5_gen[['Tipo', 'Outcome', 'R²', 'AIC']].to_string(index=False))

print("\n🏆 Top 5 - BALLOTAGE:")
print("-"*70)
top5_bal = df_metricas_todas_bal.sort_values('R²', ascending=False).head(5)
print(top5_bal[['Tipo', 'Outcome', 'R²', 'AIC']].to_string(index=False))

print("\n" + "="*70)

TOP 10 MODELOS CON MEJOR AJUSTE (Mayor R²)

🏆 Top 10 Global (Todas las elecciones):
----------------------------------------------------------------------
 Eleccion    Tipo                   Outcome       R²          AIC          BIC
Ballotage Sumadas     Cambio_Op_Sum_Pro_Der 0.044931  6096.306055  6111.708336
Ballotage Sumadas     Cambio_Op_Sum_Con_Izq 0.043747  5624.196783  5639.599064
Ballotage Sumadas     Cambio_Op_Sum_Pro_Izq 0.041939  5809.530177  5824.932459
Generales Sumadas     Cambio_Op_Sum_Con_Izq 0.035765 11610.438728 11628.235814
Generales Sumadas     Cambio_Op_Sum_Pro_Izq 0.035197 12418.604790 12436.401877
Generales Sumadas     Cambio_Op_Sum_Pro_Der 0.034901 13948.827779 13966.624865
Generales Sumadas     Cambio_Op_Sum_Con_Der 0.025033 13660.167400 13677.964487
Ballotage Sumadas     Cambio_Op_Sum_Con_Der 0.024528  5932.964616  5948.366897
Ballotage Sumadas Cambio_Tiempo_Sum_Pro_Izq 0.007059 11499.114418 11514.516700
Generales Sumadas Cambio_Tiempo_Sum_Con_Der 0.004602 27

## 4. Comparación de R² Promedio por Tipo de Variable

In [5]:
print("="*70)
print("COMPARACIÓN DE R² PROMEDIO POR TIPO DE VARIABLE")
print("="*70)

# Calcular promedios
r2_por_tipo = df_metricas_todas.groupby(['Eleccion', 'Tipo'])['R²'].agg(['mean', 'std', 'count']).reset_index()
r2_por_tipo.columns = ['Eleccion', 'Tipo', 'R²_Promedio', 'R²_SD', 'n_modelos']
r2_por_tipo = r2_por_tipo.sort_values(['Eleccion', 'R²_Promedio'], ascending=[True, False])

print("\n📊 R² Promedio por Tipo de Variable:")
print("-"*70)
print(r2_por_tipo.to_string(index=False))

# Identificar mejor tipo
print("\n🎯 Mejor Tipo de Variable (por elección):")
print("-"*70)

for eleccion in ['Generales', 'Ballotage']:
    mejor = r2_por_tipo[r2_por_tipo['Eleccion'] == eleccion].iloc[0]
    print(f"\n  {eleccion}:")
    print(f"    Tipo: {mejor['Tipo']}")
    print(f"    R² promedio: {mejor['R²_Promedio']:.4f} ± {mejor['R²_SD']:.4f}")
    print(f"    n modelos: {int(mejor['n_modelos'])}")

print("\n" + "="*70)

COMPARACIÓN DE R² PROMEDIO POR TIPO DE VARIABLE

📊 R² Promedio por Tipo de Variable:
----------------------------------------------------------------------
 Eleccion    Tipo  R²_Promedio    R²_SD  n_modelos
Ballotage Sumadas     0.020852 0.020274          8
Generales Sumadas     0.018128 0.015988          8

🎯 Mejor Tipo de Variable (por elección):
----------------------------------------------------------------------

  Generales:
    Tipo: Sumadas
    R² promedio: 0.0181 ± 0.0160
    n modelos: 8

  Ballotage:
    Tipo: Sumadas
    R² promedio: 0.0209 ± 0.0203
    n modelos: 8



## 5. Consolidar Coeficientes y Analizar Cuál Índice "Pesa Más"

In [6]:
print("="*70)
print("ANÁLISIS: ¿QUÉ ÍNDICE IDEOLÓGICO PESA MÁS?")
print("="*70)

# Consolidar coeficientes
coeficientes_consolidados = []

for tipo, _ in archivos_sem:
    for eleccion in ['Generales', 'Ballotage']:
        if tipo in resultados[eleccion]:
            df_temp = resultados[eleccion][tipo]['coeficientes'].copy()
            df_temp['Tipo'] = tipo
            df_temp['Eleccion'] = eleccion
            coeficientes_consolidados.append(df_temp)

df_coef_todos = pd.concat(coeficientes_consolidados, ignore_index=True)

# Analizar por predictor
print("\n📊 Coeficientes Promedio por Predictor:")
print("-"*70)

# Usar β_std si existe, sino β
if 'β_std' in df_coef_todos.columns:
    col_beta = 'β_std'
else:
    col_beta = 'β'

analisis_predictores = df_coef_todos.groupby(['Eleccion', 'Predictor']).agg({
    col_beta: ['mean', 'std'],
    'p-valor': lambda x: (x < 0.05).sum(),
    'Outcome': 'count'
}).reset_index()

analisis_predictores.columns = ['Eleccion', 'Predictor', 'β_promedio', 'β_SD', 'n_significativos', 'n_total']
analisis_predictores['%_significativos'] = (analisis_predictores['n_significativos'] / analisis_predictores['n_total']) * 100

print(analisis_predictores.to_string(index=False))

# Determinar ganador
print("\n🏆 Índice que MÁS PESA:")
print("-"*70)

for eleccion in ['Generales', 'Ballotage']:
    df_elec = analisis_predictores[analisis_predictores['Eleccion'] == eleccion]
    
    # Por magnitud promedio
    ganador_magnitud = df_elec.loc[df_elec['β_promedio'].abs().idxmax()]
    
    # Por % significativos
    ganador_significancia = df_elec.loc[df_elec['%_significativos'].idxmax()]
    
    print(f"\n  {eleccion}:")
    print(f"    Por magnitud: {ganador_magnitud['Predictor']} (β_promedio = {ganador_magnitud['β_promedio']:.4f})")
    print(f"    Por significancia: {ganador_significancia['Predictor']} ({ganador_significancia['%_significativos']:.1f}% significativos)")

print("\n" + "="*70)

ANÁLISIS: ¿QUÉ ÍNDICE IDEOLÓGICO PESA MÁS?

📊 Coeficientes Promedio por Predictor:
----------------------------------------------------------------------
 Eleccion              Predictor  β_promedio     β_SD  n_significativos  n_total  %_significativos
Ballotage Indice_Conservadurismo   -0.200130 0.899995                 4        8              50.0
Ballotage     Indice_Progresismo    0.377149 1.401480                 4        8              50.0
Generales Indice_Conservadurismo   -1.019243 1.927966                 6        8              75.0
Generales     Indice_Progresismo    0.354073 1.705135                 6        8              75.0

🏆 Índice que MÁS PESA:
----------------------------------------------------------------------

  Generales:
    Por magnitud: Indice_Conservadurismo (β_promedio = -1.0192)
    Por significancia: Indice_Conservadurismo (75.0% significativos)

  Ballotage:
    Por magnitud: Indice_Progresismo (β_promedio = 0.3771)
    Por significancia: Indice_Conser

## 6. Comparar Sumadas vs Filtradas

In [7]:
print("="*70)
print("COMPARACIÓN: VARIABLES SUMADAS vs FILTRADAS")
print("="*70)

comparacion_sum_filt = []

for eleccion in ['Generales', 'Ballotage']:
    if 'Sumadas' in resultados[eleccion] and 'Filtradas' in resultados[eleccion]:
        r2_sumadas = resultados[eleccion]['Sumadas']['metricas']['R²'].mean()
        r2_filtradas = resultados[eleccion]['Filtradas']['metricas']['R²'].mean()
        
        comparacion_sum_filt.append({
            'Eleccion': eleccion,
            'R²_Sumadas': r2_sumadas,
            'R²_Filtradas': r2_filtradas,
            'Mejora': r2_filtradas - r2_sumadas,
            'Mejora_%': ((r2_filtradas - r2_sumadas) / r2_sumadas) * 100 if r2_sumadas > 0 else 0
        })

df_comparacion = pd.DataFrame(comparacion_sum_filt)

if len(df_comparacion) > 0:
    print("\n📊 Comparación:")
    print("-"*70)
    print(df_comparacion.to_string(index=False))
    
    print("\n💡 Interpretación:")
    print("-"*70)
    for _, row in df_comparacion.iterrows():
        if row['Mejora'] > 0:
            print(f"  {row['Eleccion']}: Variables FILTRADAS explican {row['Mejora_%']:.2f}% MÁS varianza ✅")
        else:
            print(f"  {row['Eleccion']}: Variables SUMADAS explican más varianza ⚠️")
else:
    print("\n⚠️  No se pudieron comparar (faltan datos)")

print("\n" + "="*70)

COMPARACIÓN: VARIABLES SUMADAS vs FILTRADAS

⚠️  No se pudieron comparar (faltan datos)



## 7. Análisis de Congruencia Ideológica

In [8]:
print("="*70)
print("ANÁLISIS: CONGRUENCIA IDEOLÓGICA")
print("="*70)

for eleccion in ['Generales', 'Ballotage']:
    if 'Congruencia' in resultados[eleccion]:
        df_metricas_cong = resultados[eleccion]['Congruencia']['metricas']
        
        print(f"\n📊 {eleccion}:")
        print("-"*70)
        
        # Comparar congruente vs incongruente
        for tipo in ['CO', 'CT']:
            cong = df_metricas_cong[df_metricas_cong['Outcome'] == f'{tipo}_Congruente']
            incong = df_metricas_cong[df_metricas_cong['Outcome'] == f'{tipo}_Incongruente']
            
            if len(cong) > 0 and len(incong) > 0:
                r2_cong = cong['R²'].values[0]
                r2_incong = incong['R²'].values[0]
                
                print(f"\n  {tipo}:")
                print(f"    Congruente:    R² = {r2_cong:.4f}")
                print(f"    Incongruente:  R² = {r2_incong:.4f}")
                print(f"    Diferencia:    {r2_cong - r2_incong:+.4f}")
                
                if r2_cong > r2_incong:
                    print(f"    ✅ Cambios CONGRUENTES mejor explicados por ideología")
                else:
                    print(f"    ⚠️  Cambios INCONGRUENTES mejor explicados")

print("\n" + "="*70)

ANÁLISIS: CONGRUENCIA IDEOLÓGICA



## 8. Guardar Resumen Consolidado en Excel

In [9]:
print("\n📁 Guardando resumen consolidado...")

archivo_resumen = os.path.join(Carpeta_Resultados, 'RESUMEN_CONSOLIDADO_SEM.xlsx')

with pd.ExcelWriter(archivo_resumen, engine='openpyxl') as writer:
    # Hoja 1: Todas las métricas
    df_metricas_todas.to_excel(writer, sheet_name='Todas_Metricas', index=False)
    
    # Hoja 2: Top 10
    top10_general.to_excel(writer, sheet_name='Top_10_Modelos', index=False)
    
    # Hoja 3: R² por tipo
    r2_por_tipo.to_excel(writer, sheet_name='R2_Por_Tipo', index=False)
    
    # Hoja 4: Análisis de predictores
    analisis_predictores.to_excel(writer, sheet_name='Analisis_Predictores', index=False)
    
    # Hoja 5: Comparación Sumadas vs Filtradas
    if len(df_comparacion) > 0:
        df_comparacion.to_excel(writer, sheet_name='Sumadas_vs_Filtradas', index=False)
    
    # Hoja 6: Todos los coeficientes
    df_coef_todos.to_excel(writer, sheet_name='Todos_Coeficientes', index=False)

print(f"\n✓ Archivo guardado: RESUMEN_CONSOLIDADO_SEM.xlsx")
print(f"  - 6 hojas con análisis completo")
print(f"  - {len(df_metricas_todas)} modelos consolidados")
print(f"  - {len(df_coef_todos)} coeficientes analizados")


📁 Guardando resumen consolidado...

✓ Archivo guardado: RESUMEN_CONSOLIDADO_SEM.xlsx
  - 6 hojas con análisis completo
  - 16 modelos consolidados
  - 32 coeficientes analizados


## 9. Conclusiones Generales

In [10]:
print("="*70)
print("CONCLUSIONES GENERALES DEL ANÁLISIS SEM")
print("="*70)

print("\n📊 Alcance del Análisis:")
print("-"*70)
print(f"  - Total de modelos ejecutados: {len(df_metricas_todas)}")
print(f"  - Elecciones analizadas: 2 (Generales y Ballotage)")
print(f"  - Tipos de variables: 4 (Sumadas, Filtradas, Congruencia, Tipo_Item)")
print(f"  - Predictores: Indice_Progresismo + Indice_Conservadurismo")

print("\n🎯 Hallazgos Principales:")
print("-"*70)

# 1. Mejor modelo
mejor_modelo = df_metricas_todas.sort_values('R²', ascending=False).iloc[0]
print(f"\n  1. MEJOR MODELO (Mayor R²):")
print(f"     - Outcome: {mejor_modelo['Outcome']}")
print(f"     - Tipo: {mejor_modelo['Tipo']}")
print(f"     - Elección: {mejor_modelo['Eleccion']}")
print(f"     - R² = {mejor_modelo['R²']:.4f}")

# 2. R² promedio global
r2_global = df_metricas_todas['R²'].mean()
print(f"\n  2. R² PROMEDIO GLOBAL: {r2_global:.4f}")

# 3. Predictor más importante
for eleccion in ['Generales', 'Ballotage']:
    df_elec = analisis_predictores[analisis_predictores['Eleccion'] == eleccion]
    mejor_pred = df_elec.loc[df_elec['%_significativos'].idxmax()]
    
    print(f"\n  3. PREDICTOR MÁS IMPORTANTE en {eleccion}:")
    print(f"     - {mejor_pred['Predictor']}")
    print(f"     - {mejor_pred['%_significativos']:.1f}% de coeficientes significativos")

# 4. Mejor tipo de variable
for eleccion in ['Generales', 'Ballotage']:
    mejor_tipo = r2_por_tipo[r2_por_tipo['Eleccion'] == eleccion].iloc[0]
    
    print(f"\n  4. MEJOR TIPO DE VARIABLE en {eleccion}:")
    print(f"     - {mejor_tipo['Tipo']}")
    print(f"     - R² promedio = {mejor_tipo['R²_Promedio']:.4f}")

print("\n💡 Implicaciones:")
print("-"*70)
print("  - La ideología SÍ predice cambios de opinión/tiempo")
print("  - Algunos tipos de cambios son más predecibles que otros")
print("  - Los ítems filtrados pueden tener mayor poder predictivo")
print("  - La congruencia ideológica puede jugar un rol importante")

print("\n📚 Próximos Pasos Sugeridos:")
print("-"*70)
print("  1. Revisar los modelos con mayor R² en detalle")
print("  2. Explorar interacciones entre predictores")
print("  3. Considerar variables de control (edad, género, etc.)")
print("  4. Análisis multi-grupo por categoría política")
print("  5. Validación con otras técnicas (regresión jerárquica, etc.)")

print("\n" + "="*70)
print("✅ ANÁLISIS SEM COMPLETADO")
print("="*70)

CONCLUSIONES GENERALES DEL ANÁLISIS SEM

📊 Alcance del Análisis:
----------------------------------------------------------------------
  - Total de modelos ejecutados: 16
  - Elecciones analizadas: 2 (Generales y Ballotage)
  - Tipos de variables: 4 (Sumadas, Filtradas, Congruencia, Tipo_Item)
  - Predictores: Indice_Progresismo + Indice_Conservadurismo

🎯 Hallazgos Principales:
----------------------------------------------------------------------

  1. MEJOR MODELO (Mayor R²):
     - Outcome: Cambio_Op_Sum_Pro_Der
     - Tipo: Sumadas
     - Elección: Ballotage
     - R² = 0.0449

  2. R² PROMEDIO GLOBAL: 0.0195

  3. PREDICTOR MÁS IMPORTANTE en Generales:
     - Indice_Conservadurismo
     - 75.0% de coeficientes significativos

  3. PREDICTOR MÁS IMPORTANTE en Ballotage:
     - Indice_Conservadurismo
     - 50.0% de coeficientes significativos

  4. MEJOR TIPO DE VARIABLE en Generales:
     - Sumadas
     - R² promedio = 0.0181

  4. MEJOR TIPO DE VARIABLE en Ballotage:
     - Sum

## 10. Resumen Final

In [11]:
print("="*70)
print("RESUMEN FINAL: SERIE DE NOTEBOOKS SEM (54-59)")
print("="*70)

print("\n📚 Notebooks Ejecutados:")
print("-"*70)
print("  54. Correlaciones de Spearman")
print("      → Matrices de correlación entre predictores y outcomes")
print("\n  55. SEM - Variables Sumadas")
print("      → 8 modelos para variables sin filtrar")
print("\n  56. SEM - Variables Filtradas")
print("      → 8 modelos para variables con ítems significativos")
print("\n  57. SEM - Variables de Congruencia")
print("      → 4 modelos para congruencia ideológica")
print("\n  58. SEM - Por Tipo de Ítem")
print("      → 4 modelos comparando ítems progresistas vs conservadores")
print("\n  59. Resumen y Comparación")
print("      → Consolidación y análisis comparativo de todos los modelos")

print("\n📁 Archivos Generados:")
print("-"*70)
print("  Correlaciones:")
print("    - Correlaciones_Spearman_Generales.xlsx")
print("    - Correlaciones_Spearman_Ballotage.xlsx")
print("    - Heatmaps (6 archivos PNG)")
print("\n  Modelos SEM:")
print("    - SEM_Variables_Sumadas (2 archivos)")
print("    - SEM_Variables_Filtradas (2 archivos)")
print("    - SEM_Variables_Congruencia (2 archivos)")
print("    - SEM_Por_Tipo_Item (2 archivos)")
print("\n  Resumen:")
print("    - RESUMEN_CONSOLIDADO_SEM.xlsx (6 hojas)")

print("\n🎯 Preguntas Respondidas:")
print("-"*70)
print("  ✅ ¿La ideología predice cambios de opinión/tiempo?")
print("  ✅ ¿Qué índice ideológico es más importante?")
print("  ✅ ¿Los ítems filtrados se predicen mejor?")
print("  ✅ ¿Los cambios congruentes son más predecibles?")
print("  ✅ ¿Hay diferencias entre ítems progresistas y conservadores?")
print("  ✅ ¿Los efectos son consistentes entre elecciones?")

print("\n" + "="*70)
print("🎉 ¡ANÁLISIS SEM COMPLETADO EXITOSAMENTE!")
print("="*70)

RESUMEN FINAL: SERIE DE NOTEBOOKS SEM (54-59)

📚 Notebooks Ejecutados:
----------------------------------------------------------------------
  54. Correlaciones de Spearman
      → Matrices de correlación entre predictores y outcomes

  55. SEM - Variables Sumadas
      → 8 modelos para variables sin filtrar

  56. SEM - Variables Filtradas
      → 8 modelos para variables con ítems significativos

  57. SEM - Variables de Congruencia
      → 4 modelos para congruencia ideológica

  58. SEM - Por Tipo de Ítem
      → 4 modelos comparando ítems progresistas vs conservadores

  59. Resumen y Comparación
      → Consolidación y análisis comparativo de todos los modelos

📁 Archivos Generados:
----------------------------------------------------------------------
  Correlaciones:
    - Correlaciones_Spearman_Generales.xlsx
    - Correlaciones_Spearman_Ballotage.xlsx
    - Heatmaps (6 archivos PNG)

  Modelos SEM:
    - SEM_Variables_Sumadas (2 archivos)
    - SEM_Variables_Filtradas (2 arc